# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [310]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [311]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [312]:
import os
from glob import glob

# Write your code below
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files



['../../05_src/data/prices/BXS/BXS_1993/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_1993/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_1994/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_1994/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_1995/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_1995/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_1992/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_1992/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_2008/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_2008/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_2001/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_2001/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_2006/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_2006/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_2007/part.0.parquet',
 '../../05_src/data/prices/BXS/BXS_2007/part.1.parquet',
 '../../05_src/data/prices/BXS/BXS_2000/part.0.parquet',
 '../../05_src/data/prices/BXS/

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [313]:
# Write your code below.

import pandas as pd #import pandas


In [314]:
dd_a1= dd.read_parquet(parquet_files).set_index("ticker") #read parquet files
dd_a1



,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=89,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32
ACB,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...


In [315]:
# assign lags for Close and Adj Close
dd_p1 = (
    dd_a1
    .groupby('ticker', group_keys = False)
    .apply(
        lambda x: x.sort_values('Date', ascending = True)
            .assign(
                Close_lag_1 = x['Close'].shift(1),
                Adj_Close_lag_1 = x['Adj Close'].shift(1)),
         meta = pd.DataFrame(data = {'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
        ))

In [316]:
# Calculate returns
dd_p2 = dd_p1.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

In [317]:
# Calculate hi_low_range 
dd_feat = dd_p2.assign(
    hi_low_range = lambda x: x['High'] - x['Low']
)

In [319]:
#check dask dataframe 
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_low_range
npartitions=89,,,,,,,,,,,,,
A,object,object,object,object,object,object,object,object,object,object,object,object,object
ACB,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [308]:
# Write your code below.
dd_feat.compute() #Exculate data  



,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_low_range
ticker,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,3.160000,2.894159,2.034810,0.679999
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,3.410000,3.123127,1.551320,0.630000
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,3.430000,3.141444,1.822157,1.060000


In [326]:
#rolling mean of returns for 10 days.  Assumption: calculation up to 10 days of data available. If it has less than 10 days, calculate mean for available days.

dd_final = (
    dd_feat
    .groupby('ticker', group_keys = False)
    .apply(
        lambda x: x.sort_values('Date', ascending = True)
            .assign
            (rolling_mean = x['returns'].rolling(window=10, min_periods=1).mean()),
    meta = dd_feat._meta.assign(rolling_mean = 'f8')
))
dd_final.compute()


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_low_range,rolling_mean
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,43.750000,42.261578,-0.339873,2.280043,-0.339873
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,44.470001,42.957081,-0.292252,2.816525,-0.316063
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,44.419998,42.908775,-0.355869,2.592991,-0.329332
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,45.480000,43.932716,-0.354171,1.385908,-0.335541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2020-03-26,9.610000,9.940000,9.260000,9.590000,9.590000,60500.0,ZEUS.csv,2020,3.160000,2.894159,2.034810,0.679999,1.935249
ZEUS,2020-03-27,9.330000,9.330000,8.700000,8.700000,8.700000,52900.0,ZEUS.csv,2020,3.410000,3.123127,1.551320,0.630000,1.914275
ZEUS,2020-03-30,8.810000,9.760000,8.700000,9.680000,9.680000,73700.0,ZEUS.csv,2020,3.430000,3.141444,1.822157,1.060000,1.898678


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it’s not necessary to convert to pandas to calculate the moving average. The advantage of using Dask over pandas is that it uses less memory, and it’s better for large datasets because it does lazy execution.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.